In [20]:
import requests
from bs4 import BeautifulSoup as bs
# import pandas as pd
import json

In [24]:
json_path = "/Users/fedor/Desktop/meta_project/stop_game"
def parse_stop_game_page(games_rewievs, page_count):
    """
    Parses pages of StopGame.ru for comments of users. Gets name of the game, text of comment, score,
    lists of pluses and minuses (it's special field on a site). Stores all data in .json format.
    :param games_rewievs: list of bs4.element.Tag - list with HTML of pages
    :param page_count: int - counter that helps to give a proper names
    :return: page_count for future usage
    """
    rewiev_count = 0
    for i in range(len(games_rewievs)):
        # name
        name = games_rewievs[i].div.div.text
        # text of comment
        text = ""
        for section in games_rewievs[i].div.find_all("p"):
            text += section.text.replace("&nbsp;","")

        # score
        int_part = len(games_rewievs[i].find(class_ = "_stars_crdtt_100 _stars--big_crdtt_367").div.find_all(href="#fa/star"))

        half = 0.5*len(games_rewievs[i].find(class_ = "_stars_crdtt_100 _stars--big_crdtt_367").div.find_all(href="#fa/star-half"))
        score = int_part + half

        # pluses and minuses
        pluses = []
        minuses = []
        for comment in games_rewievs[i].find_all("span"):
            if comment.find(href="#sg/plus-thin") is not None:
                pluses.append(comment.text)
            if comment.find(href="#sg/minus-thin") is not None:
                minuses.append(comment.text)

        # date
        date = games_rewievs[i].find(class_="_info_crdtt_191").span.text.replace("Сегодня", "03.02.23").replace("Вчера", "02.02.23")


        with open(json_path+f"/page{page_count}_comment{rewiev_count}.json", 'w') as f:
            json.dump({"name": name,"text of comment": text, "score": score,
                       "pluses": pluses, "minuses": minuses, "date": date}, f)
        rewiev_count += 1
    page_count += 1
    return page_count

In [25]:
# first page have diffrent url
page_count = 0
URL_TEMPLATE = "https://stopgame.ru/games/reviews"
r = requests.get(URL_TEMPLATE)
soup = bs(r.text, "html.parser")
games_rewievs = soup.find_all(class_="_content_crdtt_63")
page_count = parse_stop_game_page(games_rewievs, page_count)

In [28]:
# url for other pages
paths = []
for i in range (2,115):
    paths.append(URL_TEMPLATE+"?page="+str(i))

In [29]:
# main loop
page_count = 1
for url in paths:
    r = requests.get(url)
    soup = bs(r.text, "html.parser")
    games_rewievs = soup.find_all(class_="_content_crdtt_63")
    page_count = parse_stop_game_page(games_rewievs, page_count)

In [30]:
# check
with open('/Users/fedor/Desktop/meta_project/stop_game/page112_comment8.json') as f:
    templates = json.load(f)
templates

{'name': 'Resident Evil 2 Remake',
 'text of comment': '\xa0 \xa0 \xa0 Мой компьютер говорит мне: "Дай глотнуть воздуха, я минималку еле держу". Однако эти сопли я вытираю достаточно быстро и уже с двумя полигонами я иду убивать не только зомбей, как Вы уже понимаете.Писать про саму игру нет абсолютно никакого смысла. Все знают, что 2018 год задал эталон всем ремейкам, представив данную игру.Хотелось бы сделать пару независимых от оценок придирок к уровню сложности игры. Мне показалось, что не хватило 4-го уровня сложности "Реализм", где ходячий Джентельмен в любом случае догонял бы протагониста, а предметы поменялись местами куда хаотичней, при дополнительном сокращении подсумков на 8, а не на 4.Дополнения все идеальны, по канону. Многие умудряются сопротивляться идее выучивания алгоритма правильного прохождения, но, очевидно, им не дано понять, что в этом весь смысл.\xa0Таким образом, игра MUST HAVE получает от меня все свои 10/10\xa0\xa0',
 'score': 5.0,
 'pluses': ['Эталон любого р

In [ ]:
# просмотр всех файлов в папке
from pathlib import Path
game_count = 0
r_count = 0

# assign directory
directory = "/Users/fedor/Desktop/meta_project/data/metacritic/user_review"


# iterate over files in
# that directory
names = Path(directory).glob('*')
for n in names:
    platforms = Path(n).glob('*')
    game_count += 1
    for p in platforms:
        rev = Path(p).glob('*')
        for _ in rev:
            r_count += 1
game_count, r_count